<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>SQL -2</center></span>

<center>update, insert into, create view</center>

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Crea una nueva base de datos](#Crea-una-nueva-base-de-datos)
* [Creación de tablas](#Creación-de-tablas)
* [Subida de datos desde un archivo externo](#Subida-de-datos-desde-un-archivo-externo)
* [Consultas de actualización, agrupación y subconsultas](#Consultas-de-actualización,-agrupación-y-subconsultas)


## <span style="color:blue">Introducción</span>

En esta leccion aprenderemos como actualizar registros en una tabla, subir datos desde una tabla a otra y a crear vistas. El ejemplo está orientado al pre-trataminento y limpieza de datos y a su posterior publicación en una vista.

Crearemos un esquema con solamente dos tablas: PAIS Y DEPARTAMENTO. Para alimentarlas traeremos datos de otras tablas  que tienen algunos problemas. Arreglaremos los problemas y luego subiremos los datos a nustras tablas *oficiales*. Finalmente eliminamos las tablas de trabao y creamos una vista.

In [2]:
SHOW DATABASES;

Database
CLASE_INFO
GRUPOS
GRUPOS_COLOMBIA
birdwatchers
information_schema
mysql
performance_schema
rookery
sakila
test


## <span style="color:blue">Crea una nueva base de datos</span>

Vamos a creas nuestra nueva base de datos.

In [3]:
CREATE OR REPLACE DATABASE departamento
CHARACTER SET 'utf8mb4'
COLLATE 'utf8mb4_general_ci';

Query OK

In [4]:
SHOW DATABASES;

Database
CLASE_INFO
GRUPOS
GRUPOS_COLOMBIA
birdwatchers
departamento
information_schema
mysql
performance_schema
rookery
sakila


In [5]:
USE departamento;

Query OK

In [6]:
SHOW TABLES;

Query OK

## <span style="color:blue">Creación de tablas</span>

### Crea dos tablas PAIS

Nustra primera tabla PAIS será nuestra  tabla final. Creamos adicionalmente una tabla de trabajo en donde vamos a subir los datos procedentes de un archivo de tipo csv. Esta tabla se llamará EN_PAIS.

In [7]:
CREATE TABLE PAIS(
IdPai MEDIUMINT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
NomPai VARCHAR(255) UNIQUE NOT NULL,
AbrevPai VARCHAR(3) UNIQUE
)ENGINE = InnoDB;

Query OK

In [26]:
CREATE TABLE EN_PAIS(
AbrevPai VARCHAR(3),
NomPai VARCHAR(255) 
);

Query OK

In [24]:
# DROP TABLE EN_PAIS;

Query OK

In [27]:
SHOW TABLES;

Tables_in_departamento
EN_PAIS
PAIS


## <span style="color:blue">Subida de datos desde un archivo externo</span>

In [ ]:
Subimos lo datos de los paises a nuestra tabla intermedia

In [28]:
LOAD DATA
LOCAL INFILE '/home/alvaro/Documents/Cursos_UN/Cursos_2021_02/Sistemas_Informacion/Datos/EN_PAIS.csv'
INTO TABLE EN_PAIS
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

Stage: 2 of 2 'End bulk insert' 0% of stage done

In [30]:
SELECT * FROM EN_PAIS
LIMIT 10;

AbrevPai,NomPai
AFG,Afganistán
AFS,Sudáfrica
AHL,Antillas Holandesas
ALB,Albania
ANB,Antigua y Barbuda
AND,Andorra
ANG,Angola
ARA,Arabia Saudita
ARG,Argentina
ARL,Argelia


### Cuenta los países

In [31]:
SELECT COUNT(*) AS TOTAL FROM EN_PAIS;

TOTAL
174


### Elimina un país

In [32]:
DELETE FROM EN_PAIS
WHERE AbrevPai ="VEN";

Query OK

In [33]:
SELECT COUNT(*) AS TOTAL FROM EN_PAIS;

TOTAL
173


### Inserta de nuevo el país

In [34]:
INSERT INTO EN_PAIS(AbrevPai, NomPai)
VALUES("VEN", "Venezuela");

Query OK

In [35]:
SELECT * FROM EN_PAIS
WHERE NomPai="Venezuela";

AbrevPai,NomPai
VEN,Venezuela


## <span style="color:blue">Subida de datos desde otra tabla</span>

Estamos listos para alimentar nuestra *tabla maestra* PAI desde la tabla EN_PAIS.

In [36]:
INSERT INTO PAIS(NomPai, AbrevPai)
SELECT NomPai, AbrevPai FROM EN_PAIS;


Query OK

### Vamos a subir de nuevo los registros pero ordenados

In [ ]:
Para reiniciar el auto numérico eliminamos la tabla y la volvemosa crear

In [39]:
DROP TABLE PAIS;

Query OK

In [40]:
SHOW TABLES;

Tables_in_departamento
EN_PAIS


In [41]:
CREATE TABLE PAIS(
IdPai MEDIUMINT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
NomPai VARCHAR(255) UNIQUE NOT NULL,
AbrevPai VARCHAR(3) UNIQUE
)ENGINE = InnoDB;

Query OK

In [42]:
INSERT INTO PAIS(NomPai, AbrevPai)
SELECT NomPai, AbrevPai FROM EN_PAIS
ORDER BY NomPai ASC;

Query OK

In [43]:
SELECT * FROM PAIS
LIMIT 10;

IdPai,NomPai,AbrevPai
1,Afganistán,AFG
2,Albania,ALB
3,Alemania,RFA
4,Andorra,AND
5,Angola,ANG
6,Antártica,ATC
7,Antigua y Barbuda,ANB
8,Antillas Holandesas,AHL
9,Arabia Saudita,ARA
10,Argelia,ARL


In [44]:
SELECT COUNT(*) AS TOTAL FROM PAIS;


TOTAL
174


## <span style="color:blue">Consultas de actualización, agrupación y subconsultas</span>

### Crea dos tablas DEPARTAMENTO

Como antes, creamos la tabla formal del sistema y una tabla de trabajo para subir dastos desde un archivo de tipo csv.

In [45]:
CREATE TABLE DEPARTAMENTO(
  IdDep MEDIUMINT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY,
  NomDep VARCHAR(100) NOT NULL,
  AbrevDep VARCHAR(3) NOT NULL,
  IdPai MEDIUMINT UNSIGNED NOT NULL,
  CONSTRAINT
    fk_DEPARTAMENTO_PAIS FOREIGN KEY (IdPai) REFERENCES PAIS (IdPai)
    ON DELETE CASCADE
    ON UPDATE RESTRICT
) ENGINE = InnoDB;

Query OK

In [46]:
CREATE TABLE EN_DEPARTAMENTO(
AbrevPai VARCHAR(3),
AbrevDep VARCHAR(3),
AbrevReg VARCHAR(3),    
NomDep VARCHAR(255),
NorDep VARCHAR(255)
)ENGINE = InnoDB;

Query OK

In [ ]:
### Subimos los datos a la tabla EN_DEPARTAMENTO y los revisamos.

In [47]:
LOAD DATA
LOCAL INFILE '/home/alvaro/Documents/Cursos_UN/Cursos_2021_02/Sistemas_Informacion/Datos/EN_DEPARTAMENTO.csv'
INTO TABLE EN_DEPARTAMENTO
FIELDS TERMINATED BY ';'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

Stage: 2 of 2 'End bulk insert' 0% of stage done

In [48]:
SELECT COUNT(*) FROM EN_DEPARTAMENTO;

COUNT(*)
310


In [49]:
SELECT * FROM EN_DEPARTAMENTO
LIMIT 20;

AbrevPai,AbrevDep,AbrevReg,NomDep,NorDep
AFG,1,,Badahsan,BADAHSAN
AFG,10,,Gazni,GAZNI
AFG,11,,Hawst,HAWST
AFG,13,,Herat,HERAT
AFG,18,,Kunar,KUNAR
AFG,8,,Faryab,FARYAB
AFG,N/A,,No Aplica,NO APLICA
AFS,N/A,,No Aplica,NO APLICA
AHL,N/A,,No Aplica,NO APLICA
ALB,3,,Devoll,DEVOLL


### Eliminamos registros que tiene N/A en AbrevDep

In [50]:
DELETE FROM EN_DEPARTAMENTO
WHERE AbrevDep="N/A";

Query OK

In [51]:
SELECT COUNT(*) AS cuenta FROM EN_DEPARTAMENTO;

cuenta
185


In [53]:
SELECT * FROM  EN_DEPARTAMENTO;

AbrevPai,AbrevDep,AbrevReg,NomDep,NorDep
AFG,1,,Badahsan,BADAHSAN
AFG,10,,Gazni,GAZNI
AFG,11,,Hawst,HAWST
AFG,13,,Herat,HERAT
AFG,18,,Kunar,KUNAR
AFG,8,,Faryab,FARYAB
ALB,3,,Devoll,DEVOLL
BRA,AL,,Alagoas,ALAGOAS
BRA,AM,,Amazonas,AMAZONAS
BRA,BA,,Bahia,BAHIA


### Modifica un registro

In [54]:
DESCRIBE EN_DEPARTAMENTO;

Field,Type,Null,Key,Default,Extra
AbrevPai,varchar(3),YES,,NULL,
AbrevDep,varchar(3),YES,,NULL,
AbrevReg,varchar(3),YES,,NULL,
NomDep,varchar(255),YES,,NULL,
NorDep,varchar(255),YES,,NULL,


In [ ]:
Quremos modificar en nombre NomDep para el registro ue contiene NorDep ="WEIBENBURG GUNZENHAUSEN".

In [57]:
UPDATE EN_DEPARTAMENTO
SET NomDep = "Weibenburg-Gunzenhausen"
WHERE NorDep = "WEIBENBURG GUNZENHAUSEN";

Query OK

In [58]:
SELECT * FROM EN_DEPARTAMENTO
WHERE NorDep = "WEIBENBURG GUNZENHAUSEN";

AbrevPai,AbrevDep,AbrevReg,NomDep,NorDep
RFA,41,BA,Weibenburg-Gunzenhausen,WEIBENBURG GUNZENHAUSEN


### Consulta de agregación

Vamos a contar el número de Departamentos por país.

In [60]:
SELECT AbrevPai, COUNT(AbrevDep)AS cuenta FROM EN_DEPARTAMENTO
GROUP BY AbrevPai;

AbrevPai,cuenta
AFG,6
ALB,1
BRA,25
CHL,13
COL,33
CUB,14
MEX,31
PAN,5
RFA,38
VEN,19


Traemos los conteos en donde hay mas de un departamento ordenados descendente por cuenta-

In [61]:
SELECT AbrevPai, COUNT(AbrevDep)AS cuenta FROM EN_DEPARTAMENTO
GROUP BY AbrevPai
HAVING cuenta>1
ORDER BY cuenta DESC;

AbrevPai,cuenta
RFA,38
COL,33
MEX,31
BRA,25
VEN,19
CUB,14
CHL,13
AFG,6
PAN,5


### Subconsultas

Supongamos que ahora desdeamos extraer el país con el mayor número de departamentos en la tabla EN_DEPARTAMENTO. Usaremos una subconsulta.

In [69]:
WITH conteo AS 
   ( SELECT AbrevPai, COUNT(AbrevDep)AS cuenta FROM EN_DEPARTAMENTO
    GROUP BY AbrevPai)  
SELECT  AbrevPai, cuenta FROM conteo
WHERE cuenta = (SELECT MAX(cuenta) FROM conteo);

AbrevPai,cuenta
RFA,38


### Sube los datos a la tabla Departamento

In [ ]:
Estamos listos para subir los datos.

In [70]:
INSERT INTO DEPARTAMENTO(NomDep, AbrevDep, IdPai)
SELECT D.NomDep,D.AbrevDep, P.IdPai
FROM PAIS AS P 
INNER JOIN EN_DEPARTAMENTO AS D
ON P.AbrevPai = D.AbrevPai;

Query OK

In [71]:
SELECT * FROM DEPARTAMENTO;

IdDep,NomDep,AbrevDep,IdPai
1,Badahsan,1,1
2,Gazni,10,1
3,Hawst,11,1
4,Herat,13,1
5,Kunar,18,1
6,Faryab,8,1
7,Devoll,3,2
8,Alagoas,AL,29
9,Amazonas,AM,29
10,Bahia,BA,29


In [72]:
DESCRIBE DEPARTAMENTO;

Field,Type,Null,Key,Default,Extra
IdDep,mediumint(8) unsigned,NO,PRI,NULL,auto_increment
NomDep,varchar(100),NO,,NULL,
AbrevDep,varchar(3),NO,,NULL,
IdPai,mediumint(8) unsigned,NO,MUL,NULL,


### Tenemos un problema

Como se exige not NULL  en el campo AbrevDep, se ha subido basura a ese campo, cunado no hay información. Vamos a alterar la extructuta de la tabla y colocaremos NULL en esos registros afectados.

Primero encontremos los registros problemáticos y luego los actualizamos

In [75]:
SELECT * FROM DEPARTAMENTO
WHERE LENGTH(AbrevDep)= 1; 

IdDep,NomDep,AbrevDep,IdPai
1,Badahsan,1,1
6,Faryab,8,1
7,Devoll,3,2
33,Tarapacá,1,41
38,Antofagasta,2,41
39,Atacama,3,41
40,Coquimbo,4,41
41,Valparaíso,5,41
42,Libertador B. O Higgins,6,41
43,Maule,7,41


In [77]:
ALTER TABLE DEPARTAMENTO MODIFY AbrevDep VARCHAR(3);

Query OK

In [78]:
DESCRIBE DEPARTAMENTO;

Field,Type,Null,Key,Default,Extra
IdDep,mediumint(8) unsigned,NO,PRI,NULL,auto_increment
NomDep,varchar(100),NO,,NULL,
AbrevDep,varchar(3),YES,,NULL,
IdPai,mediumint(8) unsigned,NO,MUL,NULL,


In [79]:
UPDATE DEPARTAMENTO
SET AbrevDep = NULL
WHERE LENGTH(AbrevDep)= 1;

Query OK

In [80]:
SELECT * FROM DEPARTAMENTO;

IdDep,NomDep,AbrevDep,IdPai
1,Badahsan,NULL,1
2,Gazni,10,1
3,Hawst,11,1
4,Herat,13,1
5,Kunar,18,1
6,Faryab,NULL,1
7,Devoll,NULL,2
8,Alagoas,AL,29
9,Amazonas,AM,29
10,Bahia,BA,29


In [ ]:
Persisten los problemas porque había núemors con dos digitos. LA priemra solución no nos sirve.

In [83]:
SELECT * FROM DEPARTAMENTO
WHERE AbrevDep REGEXP '[0-9]+$';

IdDep,NomDep,AbrevDep,IdPai
2,Gazni,10,1
3,Hawst,11,1
4,Herat,13,1
5,Kunar,18,1
34,Los lagos,10,41
35,Aysén,11,41
36,Magallanes y Ant.Chilena,12,41
37,Metropolitana de Santiago,13,41
130,Bad Tölz-Wolfratshausen,10,3
131,Baja Algovia,11,3


In [84]:
SELECT * FROM DEPARTAMENTO
WHERE AbrevDep REGEXP '^[^0-9]+$';

IdDep,NomDep,AbrevDep,IdPai
8,Alagoas,AL,29
9,Amazonas,AM,29
10,Bahia,BA,29
11,Ceará,CE,29
12,Distrito Federal,DF,29
13,Espírito Santo,ES,29
14,Goiás,GO,29
15,Maranhão,MA,29
16,Minas Gerais,MG,29
17,Mato Grosso do Sul,MS,29


In [85]:
UPDATE DEPARTAMENTO
SET AbrevDep = NULL
WHERE AbrevDep REGEXP '[0-9]+$';

Query OK

In [86]:
SELECT * FROM DEPARTAMENTO;

IdDep,NomDep,AbrevDep,IdPai
1,Badahsan,NULL,1
2,Gazni,NULL,1
3,Hawst,NULL,1
4,Herat,NULL,1
5,Kunar,NULL,1
6,Faryab,NULL,1
7,Devoll,NULL,2
8,Alagoas,AL,29
9,Amazonas,AM,29
10,Bahia,BA,29


## CREANDO VISTAS

CREATE [OR REPLACE] VIEW nombre_vista [lista_columnas]
AS consulta

Primero verificamos la consulta de origen

In [88]:
SELECT D.IdDep, D.NomDep AS Departamento, P.NomPai AS Pais
FROM PAIS AS P 
INNER JOIN DEPARTAMENTO AS D
ON P.IdPai = D.IdPai
ORDER BY Pais ASC, Departamento ASC ;

IdDep,Departamento,Pais
1,Badahsan,Afganistán
6,Faryab,Afganistán
2,Gazni,Afganistán
3,Hawst,Afganistán
4,Herat,Afganistán
5,Kunar,Afganistán
7,Devoll,Albania
129,Aichach-Friedberg,Alemania
153,Alb-Donau-Kreis,Alemania
135,Algovia Oriental,Alemania


### Crea la vista

In [90]:
CREATE OR REPLACE VIEW vDEPARTAMENTO
AS 
SELECT D.IdDep, D.NomDep AS Departamento, P.NomPai AS Pais
FROM PAIS AS P 
INNER JOIN DEPARTAMENTO AS D
ON P.IdPai = D.IdPai
ORDER BY Pais ASC, Departamento ASC ;

Query OK

In [91]:
SHOW TABLES;

Tables_in_departamento
DEPARTAMENTO
EN_DEPARTAMENTO
EN_PAIS
PAIS
vDEPARTAMENTO


### USAR LA VISTA

In [92]:
SELECT Pais, count(Departamento) as Frecuencia
FROM  vDEPARTAMENTO
GROUP BY Pais;


Pais,Frecuencia
Afganistán,6
Albania,1
Alemania,38
Brasil,25
Chile,13
Colombia,33
Cuba,14
México,31
Panamá,5
Venezuela,19


In [93]:
SELECT Pais, count(Departamento) as Frecuencia
FROM  vDEPARTAMENTO
GROUP BY Pais
HAVING Frecuencia > 12
ORDER BY FRECUENCIA DESC;

Pais,Frecuencia
Alemania,38
Colombia,33
México,31
Brasil,25
Venezuela,19
Cuba,14
Chile,13


### Verificación Colombia

In [94]:
SELECT Departamento, count(Departamento) as Frecuencia
FROM  vDEPARTAMENTO
WHERE Pais ="Colombia"
GROUP BY Departamento
ORDER BY FRECUENCIA DESC;

Departamento,Frecuencia
Arauca,1
Cordoba,1
Huila,1
Santander,1
Cesar,1
Guaviare,1
Putumayo,1
Vichada,1
Bolivar,1
Casanare,1


In [95]:
SELECT * FROM vDEPARTAMENTO
ORDER BY Pais ASC, Departamento ASC;

IdDep,Departamento,Pais
1,Badahsan,Afganistán
6,Faryab,Afganistán
2,Gazni,Afganistán
3,Hawst,Afganistán
4,Herat,Afganistán
5,Kunar,Afganistán
7,Devoll,Albania
129,Aichach-Friedberg,Alemania
153,Alb-Donau-Kreis,Alemania
135,Algovia Oriental,Alemania


In [96]:
SELECT COUNT(*) FROM vDEPARTAMENTO
WHERE Pais ="Colombia";


COUNT(*)
33


### Finalmente eliminamos las tablas de trabajo

In [98]:
DROP TABLE EN_PAIS;

Query OK

In [99]:
DROP TABLE EN_DEPARTAMENTO;

Query OK

In [100]:
SHOW TABLES;

Tables_in_departamento
DEPARTAMENTO
PAIS
vDEPARTAMENTO
